In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
from IPython.display import HTML
import scipy as sp

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.1e}'.format})

In [68]:
# Fundamental constants
h    = 6.626e-34
GHz = 1e9
e0   = 1.602e-19
Φ_0 = h/(2*e0)

# Circuit parameters
Csh = 15
C   = 15
Cg  = 10
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0
l = Lq*(Lq+4*Lr) - 4*Δ**2

Cc = 10

In [73]:
nmax_r = 20
nmax_f = 20

# Symmetric Cc coupling
That is, the inner nodes of the qubits are coupled with Cc, and the outer nodes of the qubits are coupled with Cc to their respective readout lines.
As a result, the renormalization induced by Cc ensures that the qubit and resonator modes are still the symmetric and atisymmetric combination of the fluxes of the upper nodes.

## Composing the circuit with fluxoniums and resonatrs

In [74]:
def hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc, C = 15, CJ = 3, Csh= 15, Lq = 25, Lr = 10, Δ = 0.1, coupling=1 ):
    
    resonator = sq_ext.KIT_resonator(C = C+Cc,                     Lq = Lq, Lr = Lr, Δ = Δ)
    fluxonium = sq_ext.KIT_fluxonium(C = C+Cc, CJ = CJ, Csh = Csh, Lq = Lq, Lr = Lr, Δ = Δ)
    
    resonator.set_trunc_nums([nmax_r])
    fluxonium.set_trunc_nums([nmax_f])
    
    H_left  = sq_ext.hamiltonian_frc(fluxonium, resonator,Δ)
    H_right = sq_ext.hamiltonian_frc(fluxonium, resonator,Δ)
    
    I_r       = qt.identity(nmax_r)
    I_f       = qt.identity(nmax_f)
    I_H_left  = qt.identity(H_left .dims[0])
    I_H_right = qt.identity(H_right.dims[0])
    
    q_r = qt.tensor(resonator.charge_op(0), I_f)
    q_f = qt.tensor(I_r, fluxonium.charge_op(0))
    
    if Cc>0:
        H = qt.tensor(H_left, I_H_right) + qt.tensor(I_H_left, H_right) + 1/Cc * (qt.tensor(q_r,q_r) + 
                                                                                  qt.tensor(q_f,q_f) + 
                                                                                  qt.tensor(q_r,q_f) + 
                                                                                  qt.tensor(q_f,q_r) ) * coupling
    else:
        H = qt.tensor(H_left, I_H_right) + qt.tensor(I_H_left, H_right)
        
    return H


In [75]:
H_0 = hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc=Cc, coupling=0)
sq_ext.diag(H_0, n_eig=6, out='GHz')[0]

array([13.730211767148, 15.353256627126, 15.353256627126, 16.976301487104, 19.090348890134, 19.090348890135])

In [76]:
H   = hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc=Cc)
sq_ext.diag(H, n_eig=6, out='GHz')[0]

array([13.730211767148, 15.353256627126, 15.353256627127, 16.976301487104, 19.090348890134, 19.090348890134])

## Creating the full circuits with sqcircuits to check that the energies are the same

In [39]:
# Initialize loops
loop_qubit = sq.Loop(0.5)
loop1_qubit_C_qubit= sq.Loop(0.5)
loop2_qubit_C_qubit = sq.Loop(0.5)

# Circuit components
C_01, C_01_rc, C_02, C_02_rc, C_12, L_03, L_31, L_23, JJ_12 = [[] for _ in range(9)]
for loop in [loop_qubit, loop1_qubit_C_qubit, loop2_qubit_C_qubit]:
    C_01       .append(sq.Capacitor(C,          'fF'))
    C_01_rc    .append(sq.Capacitor(C+Cc,       'fF')) # With extra Cc for redout or coupling
    C_02       .append(sq.Capacitor(C,          'fF'))
    C_02_rc    .append(sq.Capacitor(C+Cc,       'fF')) # With extra Cc for redout or coupling
    C_12 .append(sq.Capacitor(CJ+Csh,  'fF'))
    L_03 .append(sq.Inductor(Lr,       'nH'))
    L_31 .append(sq.Inductor(Lq/2 - Δ, 'nH',  loops=[loop]))
    L_23 .append(sq.Inductor(Lq/2 + Δ, 'nH',  loops=[loop]))
    JJ_12.append(sq.Junction(EJ,       'GHz', loops=[loop]))
C_24 = sq.Capacitor(Cc, 'fF')


### Two uncoupled qubits with capacitances for coupling and readout

In [40]:
# Two qubits
elements_qubit_qubit = {
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[1]],
    (0, 1): [C_01_rc[1]],
    (0, 2): [C_02_rc[1]],
    (3, 1): [L_31[1]],
    (1, 2): [C_12[1], JJ_12[1]],
    (2, 3): [L_23[1]],
    # qubit 2, nodes [0, 4, 5, 6]
    (0, 6): [L_03[2]],
    (0, 4): [C_01_rc[2]],
    (0, 5): [C_02_rc[2]],
    (6, 4): [L_31[2]],
    (4, 5): [C_12[2], JJ_12[2]],
    (5, 6): [L_23[2]],
}

qubit_qubit = sq.Circuit(elements_qubit_qubit)
qubit_qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [41]:
qubit_qubit.set_trunc_nums([nmax_r, nmax_r, nmax_f, nmax_f])
_ = qubit_qubit.diag(6)

In [42]:
qubit_qubit.efreqs-qubit_qubit.efreqs[0]

array([0.000000000000, 1.908100125219, 1.908100125220, 3.816200250439, 6.067760047707, 6.067760047707])

In [51]:
E_0 = sq_ext.diag(H_0,6, out='GHz')[0]

In [44]:
E_0-E_0[0]

array([0.000000000000, 1.908099440162, 1.908099440163, 3.816198880325, 6.067754862726, 6.067754862726])

### Two qubits capacitively coupled

In [30]:
elements_qubit_C_qubit = { 
    # qubit 1, nodes [0, 1, 2, 3]
    (0, 3): [L_03[1]],
    (0, 1): [C_01_rc[1]],
    (0, 2): [C_02[1]],
    (3, 1): [L_31[1]],
    (1, 2): [C_12[1], JJ_12[1]],
    (2, 3): [L_23[1]],
    # qubit 2, nodes [0, 4, 5, 6]
    (0, 6): [L_03[2]],
    (0, 4): [C_01[2]],
    (0, 5): [C_02_rc[2]],
    (6, 4): [L_31[2]],
    (4, 5): [C_12[2], JJ_12[2]],
    (5, 6): [L_23[2]],
    # capacitive coupling
    (2, 4): [C_24]
}

qubit_C_qubit = sq.Circuit(elements_qubit_C_qubit)
qubit_C_qubit.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [31]:
qubit_C_qubit.set_trunc_nums([nmax_r, nmax_r, nmax_f, nmax_f])
_ = qubit_C_qubit.diag(6)

In [32]:
qubit_C_qubit.efreqs-qubit_C_qubit.efreqs[0]

array([0.000000000000, 1.899945395253, 1.916838577245, 3.815695110517, 6.066045699734, 6.068870504052])

In [45]:
E = sq_ext.diag(H,6, out='GHz')[0]

In [46]:
E-E[0]

array([0.000000000000, 1.908099440162, 1.908099440163, 3.816198880325, 6.067754862726, 6.067754862726])

# Effective hamiltonians

In [18]:
n_eig = 6

In [49]:
np.round(sq_ext.H_eff_p1(H_0, H, n_eig),4)

array([[ 1.34492e+01+0.j    ,  0.00000e+00+0.j    , -0.00000e+00+0.j    , -0.00000e+00+0.j    , -0.00000e+00-0.j    ,  0.00000e+00+0.j    ],
       [ 0.00000e+00-0.j    ,  1.53573e+01+0.j    , -9.13000e-02+0.019j ,  0.00000e+00-0.j    , -0.00000e+00+0.j    ,  0.00000e+00+0.j    ],
       [-0.00000e+00-0.j    , -9.13000e-02-0.019j ,  1.53573e+01+0.j    ,  0.00000e+00+0.j    , -0.00000e+00+0.j    , -0.00000e+00+0.j    ],
       [-0.00000e+00-0.j    ,  0.00000e+00+0.j    ,  0.00000e+00-0.j    ,  1.72654e+01-0.j    ,  0.00000e+00-0.j    , -0.00000e+00+0.j    ],
       [-0.00000e+00+0.j    , -0.00000e+00-0.j    , -0.00000e+00-0.j    ,  0.00000e+00+0.j    ,  1.95169e+01-0.j    , -9.20000e-03+0.0829j],
       [ 0.00000e+00-0.j    ,  0.00000e+00-0.j    , -0.00000e+00-0.j    , -0.00000e+00-0.j    , -9.20000e-03-0.0829j,  1.95169e+01+0.j    ]])

In [50]:
np.round(sq_ext.H_eff_SWT(H_0, H, n_eig),4)

array([[13.449200000000, -0.000000000000, -0.000000000000, -0.000000000000, -0.000000000000, -0.000000000000],
       [-0.000000000000, 15.357300000000, 0.000000000000, -0.000000000000, 0.000000000000, 0.000000000000],
       [-0.000000000000, 0.000000000000, 15.357300000000, -0.000000000000, -0.000000000000, -0.000000000000],
       [-0.000000000000, -0.000000000000, -0.000000000000, 17.265400000000, 0.000000000000, 0.000000000000],
       [-0.000000000000, 0.000000000000, -0.000000000000, 0.000000000000, 19.516900000000, 0.000000000000],
       [-0.000000000000, 0.000000000000, -0.000000000000, 0.000000000000, 0.000000000000, 19.516900000000]])